[RunnableBranch: Dynamically route logic based on input](https://python.langchain.com/docs/expression_language/how_to/routing)

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import ConfigurableField

In [2]:

# - create the Anthropic Model
ChatAnthropic = Bedrock(model_id="anthropic.claude-v2:1",  model_kwargs={'max_tokens_to_sample':200})
ChatLlama = Bedrock(model_id="meta.llama2-13b-chat-v1")
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatAnthropic
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

' Anthropic'

In [3]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatAnthropic
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatAnthropic
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatAnthropic
)

In [4]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [5]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [6]:
full_chain.invoke({"question": "how do I use Anthropic?"})

' As Dario Amodei told me, to use Anthropic you first need to sign up for an account at www.anthropic.com. Then you can use one of their AI assistants like Claude or Constitutional AI through their website or API.'

In [7]:
full_chain.invoke({"question": "how do I use LangChain?"})

" As Harrison Chase told me, to use LangChain you first need to sign up for an API key. Once you have an API key, you can make requests to the LangChain API from your code. Some things you can do include:\n\n- Ask questions and get responses from the AI assistant\n- Fine-tune the assistant by providing examples of questions and answers\n- Build chatbots and other AI applications by integrating the API into your code\n\nThe main ways to interact with LangChain are through the REST API and the Python library. The REST API allows you to make requests using JSON over HTTP. The Python library wraps the API for convenience when building applications in Python. Refer to the LangChain documentation for code samples and details on using both methods.\n\nA few best practices Harrison Chase suggested:\n\n- Start with simple questions and move to more complex ones as you get familiar with LangChain's capabilities\n- Fine-tune on your specific use case for better responses relevant to"

In [8]:
full_chain.invoke({"question": "whats 2 + 2"})

' 4\n\nThe question asked what 2 + 2 equals. The sum of 2 + 2 is 4. So the answer is 4.'

In [9]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch
full_chain.invoke({"question": "how do I use Anthropic?"})

" As Dario Amodei told me, to use Anthropic you first need to sign up for an account at anthropic.com. Once you have an account, you can use their web interface or APIs to configure and interact with AI assistants.\n\nSome key things he mentioned:\n\n- Use Constitutional AI to define the assistant's purpose and operating constraints upfront, ensuring it remains helpful, harmless, and honest. This gives you more control and peace of mind.\n\n- Leverage self-supervised and reinforced learning to efficiently train assistants on the specific skills you need, while preserving safety. This focuses the assistant's capabilities on your use case.\n\n- Integrate the assistant into chat, voice, or other interfaces to put its intelligence to work in real applications. Carefully monitor its behavior at first before fully deploying.\n\n- Provide clear, constructive feedback when the assistant acts in unexpected or unhelpful ways. This continuous feedback loop enables it to improve over"

In [10]:
full_chain.invoke({"question": "how do I use LangChain?"})

' As Harrison Chase told me, to use LangChain you first need to sign up for an API key. Once you have an API key, you can make requests to the LangChain API from your code. Some ways to interact with LangChain include:\n\n- Using the Python library to make requests. You can install the library with `pip install langchain` and then make requests like:\n\n```python\nimport langchain\n\nresponse = langchain.ask("What is the capital of France?")\nprint(response)\n```\n\n- Making HTTP requests directly to the API endpoints. For example:\n\n```\ncurl https://api.langchain.com \\\n  -H "Authorization: Bearer YOUR_API_KEY" \\\n  -H "Content-Type: application/json" \\\n  -d \'{"prompt": "What is the capital of France?"}\'\n```\n\n- Using the LangChain playground UI at https://playground.lang'

In [11]:
full_chain.invoke({"question": "whats 2 + 2"})

' 4'

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatAnthropic
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

 Here is the equation written algebraically and the solution:

EQUATION: x^3 + 7 = 12
SOLUTION: 
x^3 + 7 = 12
x^3 = 12 - 7
x^3 = 5
x = (5)^(1/3)
x = 1.71


and want to call the model with certain stop words:

In [13]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))

In [14]:
llm = ChatAnthropic.configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatAnthropic,
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatAnthropic,
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm

# By default it will call Anthropic
chain.invoke({"topic": "bears"})

" Here's a silly joke about bears:\n\nWhat do you call a bear with no teeth?\nA gummy bear!"

### With Prompts
We can do a similar thing, but alternate between prompts

In [15]:
llm = ChatAnthropic
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

# By default it will write a joke
chain.invoke({"topic": "bears"})

" Why don't bears like fast food? Because they can't catch it!"

In [16]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

" Here's a short poem about bears:\n\nThe bears amble through the forest green,\nSniffing out berries and things unseen,\nClambering up trees so very high,\nWatching the world go wandering by.\nCubs tumble and play without a care, \nLearning to forage here and there.\nWhen winter comes they'll slumber deep,\nAnd dream their bear dreams so very sweet.\nThen spring will come, they'll wake anew,\nAnother year of bear things to do!"

### With Prompts and LLMs
We can also have multiple things configurable! Here’s an example doing that with both prompts and LLMs.

In [17]:
llm = ChatAnthropic.configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatLlama,
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatLlama,
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [18]:
# We can configure it write a poem with OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

'\n\nBears, oh bears, so strong and wild\nTheir fur so soft, their eyes so mild\nThey roam the forests, free and proud\nTheir growls and roars echo loud\n\nTheir claws so sharp, their power so great\nThey rule the woods, their domain so sweet\nThey play and climb, they fish and hunt\nTheir beauty leaves us all in awe and want\n\nSo let us cherish these noble beasts\nAnd do our part to keep their homes sweet\nFor they are treasures of the wild\nAnd their beauty should be preserved and smiled.'

In [19]:
# We can always just configure only one if we want
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

'.\n\nHere\'s one:\n\nWhy did the bear go to the party?\n\nBecause he heard it was a "grizzly" good time!'

In [20]:
prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

In [27]:
from langchain_core.runnables import chain

In [28]:
@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = ChatLlama.invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatAnthropic | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

In [29]:
custom_chain.invoke("bears")

" Let me know if you'd like to hear another joke. I've got quite the repertoire of hilarious animal-themed puns."